Подключаем библиотеки

In [1]:
import json
from pprint import pprint
import pandas as pd
import os
import re
from telegram import ParseMode
import tg_analytic


Подключаем телеграм

In [2]:
# !pip install python-telegram-bot --upgrade

In [3]:
import logging

from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [4]:
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger()

In [5]:
name_of_files = os.listdir('data')

In [6]:
ds = pd.read_json(f"data/{name_of_files[0]}") # We create dataframe of first element of file
for el in name_of_files[1:]:
    ds = pd.concat([ds, pd.read_json(f'data/{el}')], ignore_index=True) # we add elements in datafame except first file


# Новый раздел

In [7]:
new = ds.copy().sort_values(by = ['product_group_name','product_price'])[['product_group_name', 'product_name','name_store','product_human_volume','product_price', 'category_name']]

Уберем сокращения

In [29]:
def search(df, qstn):
    words = [word for word in qstn.split() if word.isalpha()]
    result = []
    df['product_name_for_search'] = df['product_name'].str.lower()
    
    if len(words) == 1:
        result = df[df["product_name_for_search"].str.contains(qstn.lower())][['product_name',
                                        'name_store','product_price', 'product_human_volume', 'category_name']]
        if len(result['product_name']) == 0:
            return ('Ничего не найдено. Попробуйте уточнить запрос. ')
        result = result.to_json(force_ascii=False)
        result = json.loads(result)
    else:
        first = df[df["product_name_for_search"].str.contains(words[0].lower())][['product_name',
                                        'name_store','product_price', 'product_human_volume', 'category_name', 'product_name_for_search']]
        first['product_name_for_search'] = df['product_name'].str.lower()

        for i in words[1:]:
            first = first[first["product_name_for_search"].str.contains(i.lower())][['product_name','name_store','product_price', 'product_human_volume', 'category_name', 'product_name_for_search']]
#             print(first)
            if len(first['product_name']) == 0:
                return ('1 Ничего не найдено. Попробуйте уточнить запрос.')
        first = first[['product_name','name_store','product_price', 'product_human_volume']]
        
        result = first.to_json(force_ascii=False)
        result = json.loads(result) 
    
    result = pd.DataFrame(result) 
    firsts_list = []
    
    for i in result['product_name']:
        if words[0].lower() == i.split(" ")[0].lower():
            firsts_list.append(i)
            
    res1 = result.loc[result['product_name'].isin(firsts_list)]
    res2 = result.loc[~result['product_name'].isin(firsts_list)]
    
    result = pd.concat([res1, res2], axis=0)
    result_groupby = pd.DataFrame(result).groupby(result['product_name'], sort=False).groups
    res = []
    
#     print(df.keys())
    
    for i in result_groupby:
#         new_main_category = 0
#         print(new_main_category)
        res.append(f"<b>{i}</b>:\n")
        for j in result[result['product_name'] == i]['product_name'].index:
            res.append(f"- в магазине {result['name_store'][j]} по цене {str(result['product_price'][j])} за {result['product_human_volume'][j]}\n")
#         res.append(f"Категория: {category_links[result['category_name'][j]]} \n\n")
        res.append("\n")
#     res.append("\n")
        
    
    
    return(''.join(res)[:3950])


Проверка

Вопрос - ответ

In [34]:
# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Привет. Меня зовут Сара! Я чатбот сервиса Сарафан. Я могу помочь сравнить цены на товары из разных магазинов.')

def echo(update: Update, context: CallbackContext):
    qstn = update.message.text    
    tg_analytic.statistics(update.message.from_user.id, update.message.message_id, update.message.text)
    
    try:
        update.message.reply_text(f'{search(new, qstn)}\nЗаказать можно на сайте: www.sarawan.ru\nСледить за новостями: https://t.me/sarawanru',
                                     parse_mode=ParseMode.HTML)
    except:
        update.message.reply_text(f'Ничего не найдено. Попробуйте уточнить поисковый запрос.\nЗаказать на сайте - sarawan.ru\nhttps://t.me/sarawanru')

Запускаем телеграм

In [35]:
updater = Updater("5005226612:AAHrkDfFcs9jmwo7cxTi6iCcOoSc25bCRXY", use_context=True) #Real token
# updater = Updater("5584200084:AAGnj5E7kPeowBTLBpxT4zoIZQjTQbN3wd8", use_context=True) #Test token
dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

# Start the Bot
updater.start_polling()
updater.idle()

2022-06-17 10:29:33,494 - apscheduler.scheduler - INFO - Scheduler started
2022-06-17 10:31:32,827 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2022-06-17 10:31:32,830 - apscheduler.scheduler - INFO - Scheduler has been shut down
